# 前言

![](assets/模型评估与选择_xmind.png)

# 评估

## 评估方法

1. 留出法:数据集划分为A、B 2个互斥集合，一般是7:3

2. 交叉验证：同分布划分K个集合，每次都是K-1并集训练，1个集合作为测试

3. 留1法：实际评估模型与D数据集训练的模型相似，但是一旦样本过大，基本不可用

4. 自助法：放回采样，m次采样：$\lim_{x \rightarrow \infty }(1-\frac{1}{m})^{m}=\frac{1}{e}\approx 0.368$,所以，每次都有36.8%的数据不会被采到，用$D/{D}'$作为测试集；**缺点**自助法产生的数据集改变了初始数据集的分布，只在**样本较少情况**下使用


## 调参与最终模型

对参数按照一定范围，一定步长进行迭代，如特征 $x \in [0,0.2]$按照0.05为步长，则有5个候选参数，如果有100个特征，则有$5^{100}$个模型需要考虑

关于训练集，验证集，测试集概念：<训练集，验证集>训练测试模型，模型实际使用中遇到的数据叫做测试集

# 性能度量

核心是模型的泛化能力的评价标准

预测任务中，给定$D={(x1, y1), (x2, y2),..., (x_m, y_m)}$，其中$y_i$是示例$x_i$的真实记标记，评估学习器f的性能，就是要把学习器预测结果$f(x_i)$与真实标记$y_i$进行比较


## 回归任务

回归人物最常用的性能度量指标是均方误差，

$$
E(f;D) = \frac{1}{m}\sum_{i=1}^{m}(f(x_i) - y_i)^2
$$

更一般的就表示，对于数据分布D和概率密度函数$p(.)$,均方误差可以表示为：

$$
    E(f;D) = \int_{x\sim D}^{}(f(x_i) - y_i)^2 p(x)d_x
$$

## 分类任务

错误率与精度是最常用的评估指标，其中错误率定义为：
$$
E(f;D) = \frac{1}{m}\sum_{i=1}^{m}\mathbb{I}(f(x_i) \neq y_i)
$$
精度定义为：

$$ 
    acc(f;D) = 1 - E(f;D)
$$

更一般的定义：

$$
 E(f;D) = \int_{x\sim D}^{}\mathbb{I}(f(x_i) \neq y_i) p(x)d_x
$$


### 准确率，召回率与F1值

分类结果混淆矩阵

| 真实情况 | 预测结果     | 预测结果     |
|-------- | ------------ | ------------ |
|          | 正例         | 反例         |
| 正例     | TP（真正例） | FN（假反例） |
| 反例     | FP（假正例） | TN（真反例） |

$准确率=\frac{TP}{TPFP}$

$召回率=\frac{TP}{TP + FN}$


#### P-R图与评估

![PR图](assets/PR图.png)
**P-R图**：(`按预测序值对样本排序`)X轴为召回率（0-1之间，step可以根据需要设定），Y轴为准确率，能够比较直观的比较两个模型的优劣
山寨评估方法：P-R图面积，BEF（Break-Even Point，平衡点，当准确率==召回率时的取值）值

1. F1值（第二个公式节省了很多计算时间）：

$$
    F1=\frac{2 * P * R}{P + R} = \frac{2 * TP}{样本总数 + TP - TN}
$$



2.
加权F1值，不同系统对P，R值倾向不同
$$
   F_{\beta } = \frac{(1 + \beta^2) * P * R}{(\beta^2
* P) + R}
$$
   $\beta > 1$，召回率更有影响，反之，准确率影响更大
如做交叉验证等多次训练，如何计算F1值：方法1，先计算每一次P，R，在计算P,R的平均值，最后根据公式计算F1值；方法2：计算TP，FN，FP，TN均值，在根据公式计算F1
**F1值缺点**：两个模型，一个准确率超高，一个召回率超高，可能两个模型F1值相等，达不到我们要的比较

### ROC，AUC
![ROC_AUC](assets/ROC_AUC.png)

背景：大多数模型，都是对分类结果预测一个概率，在与给定的分类阈值作比较
ROC：X轴假正例率$FPR=\frac{FP}{FP + TN}$，Y轴为$TPR=\frac{TP}{TP + FN}$
ROC曲线画法，假设$m^+$个正例与$m^-$个反例

1. 按照预测概率排序，依次设置每个样例预测值，初始点为(0, 0)，设置为最大的

2. 此时前一个坐标点为(x,y)

   1. 如果此时为真正例：$(x, y + \frac{1}{m^+})$
   2. 如果此时为假正例：$(x + \frac{1}{m^-}, y)$




#### ROC评估

计算ROC曲线下面积，AUC（Area Under ROC
Curve）,如果按照上述方法生成ROC曲线，则AUC面积为：
$$
AUC = \frac{1}{2} \sum_{i=1}^{m-1}(x_{i+1} -
x_i) \cdot (y_{i+1} + y_i)
$$

| 真实情况 | 预测结果     | 预测结果     |
| -------- |------------ | ------------ |
|          | 正例         | 反例         |
| 正例     |TP（真正例） | FN（假反例） |
| 反例     | FP（假正例） | TN（真反例）|

从曲线构造的角度，AUC考虑的是***排序质量***，误差与排序质量紧密相关

$$
\iota_{rank} = \frac{1}{m^+ \cdot m^-} \sum_{x^+ \in D^+} \sum_{x^- \in D^-}(\mathbb{I}(f(x^+) < f(x^-)) + \frac{1}{2}\mathbb{I}(f(x^+) = f(x^-)))
$$
由此可见，$l_{rank}$表示的是ROC上部的面积，及
$$
AUC = 1 - l_{rank}
$$

#### 代价敏感的错误率与代价曲线
本质上是ROC上的真正例，与假正例的的影响程度不一样，如真癌症预判为假，与假癌症预判为真影响完全不一样。

二分类代价矩阵

| 真实类别 | 预测类别| 预测类别    |
| -------- | ----------- | ----------- |
|          | 第0类       | 第1类       |
| 第0类    | 0           | $cost_{01}$ |
| 第1类    | $cost_{10}$ | 0 |

由损失函数定义可知,代价敏感的错误率为：
$$
E(f;D;cost) = \frac{1}{m}(\sum_{x^+ \in D^+} \mathbb{I}(f(x_i \neq y_i)*cost_{01} + \sum_{x^+ \in D^-} \mathbb{I}(f(x_i \neq y_i)*cost_{10})
$$

> 以上公式为何取$\frac{1}{m}$：换个角度，整体样例M个，$\frac{1}{m}\sum_{x^+ \in D} \mathbb{I}(f(x_i \neq y_i) *cost_i)$，其中$cost_i$取的值为不同的错误分类时取对应的cost
ROC是一种特殊的，代价均等的曲线，可以按照以下构造,横轴取[0, 1]的正例概率代价：
$$
p(+)cost = \frac{p * cost_{01}}{p * cost_{01} + (1 - p) * cost_{10}}
$$
，其中p表示样例为正的概率，纵轴是取值为[0,
1]的归一化代价：
$$
cost_{norm} = \frac{FNR * p * cost_{01} + FPR * (1 - p) * cost_{10}}{p * cost_{01} + (1-p) * cost_{10}}
$$
其中，FPR是假正例率，FNR=1-FPR，假反例率
![daijiacost](assets/daijiacost.png)

## 比较检验
假设检验中的”假设“是对模型泛化错误率分布的某种判断或猜测。**现实中，我们并不知道模型的泛化错误率，只知道测试错误率$\hat{ \varepsilon
}$**

也就是m个测试样本中，恰有${m}' = m*\hat{ \varepsilon
}$个被误分类,其余样本全部正确分类的概率${\varepsilon}^{{m}'}(1-\varepsilon)^{m-{m}'}$,由此，可以估算出恰好有$m*\hat{
\varepsilon }$被误分类概率如下：
$$
P(\hat{ \varepsilon }, \varepsilon) =
\binom{n}{m*\hat{ \varepsilon }}\varepsilon^{m*\hat{ \varepsilon }}(1- \varepsilon)^{m-m*\hat{ \varepsilon }}
$$
**注意**:$\varepsilon$与$\hat{
\varepsilon }$关系。

为什么可以用比较检验：求$\varepsilon$偏导，$\frac{\partial P(\hat{
\varepsilon }, \varepsilon)}{\partial \varepsilon} = 0$可知，当$\hat{ \varepsilon }
= \varepsilon$时，$P(\hat{ \varepsilon }, \varepsilon)$取最大值，当$\left | \hat{
\varepsilon } - \varepsilon \right |$增大时$\partial P(\hat{ \varepsilon
})$减小，这符合二向分布，概率公式也与二项分布同([统计基础篇之十：怎么理解二项分布](https://zhuanlan.zhihu.com/p/24692791))，n足够大，二项分布近似正态分布
$$
E(X) = np;D(x)=np(1-p)
$$


### 二项检验

举例来看，如果$\varepsilon =
0.3$则10个样本中，测得3个样本被误分类概率最大。
![binomial_distribution](assets/binomial_distribution.png)
我们可以使用"二项检验"来对$\varepsilon \leqslant
0.3$(即泛化误差率不大于0.3)这样的假设进行检验。更一般的，考虑假设$\varepsilon \leqslant
\varepsilon_0$，则在$1-\alpha $的概率内所能观测到的最大错误率做如下计算

> 这里 $1-\alpha
$反映了结论的置信度，相当于图中阴影部分

$$
\bar{\varepsilon } = max \ \varepsilon \qquad s.t.
\qquad \sum_{i=\varepsilon_0 * m +
1}^{m}\binom{m}{i}\varepsilon^i(1-\varepsilon)^{m-i}<\alpha
$$

### t检验(单模型泛化误差评估)
模型在训练测试时采用多次留出法or交叉验证，会得到多个测试误差，此时可以使用“t检验”。此时我们得到k个测试误差率$\hat{\varepsilon
}_1,\hat{\varepsilon }_2,\hat{\varepsilon }_3,...,\hat{\varepsilon
}_k,$则平均测试误差率$\mu, \sigma^2$为：
$$
\mu = \frac{1}{k}
\sum_{i=1}^{k}\hat{\varepsilon }_i \\
\sigma^2 = \frac{1}{k-1}
\sum_{i=1}^{k}(\hat{\varepsilon }_i - \mu)^2
$$
考虑到这k个测试错误率可以看作是泛化错误率$\varepsilon_0$的独立采用，则构造t统计量：
$$
\tau_t =
\frac{\sqrt{k}(\mu - \varepsilon_0)}{\sigma}
$$
服从自由度为$k-1$的t分布
![T分布](assets/T分布.png)

双边T检验，即如图，若平均错误率$\mu$与$\varepsilon_0$之差$\left | \mu -
\varepsilon \right |$位于临界范围$[t_{-1 \cdot \frac{\alpha}{2}},
t_{\frac{\alpha}{2}}]$内，则不能拒绝假设$\mu =
\varepsilon_0$，则可认为泛化错误率为$\varepsilon$,置信度为$1- \alpha$,否则拒绝.以下为常见的$\alpha$与临界值：
![T分布常规值](assets/T分布常规值.png)

### 交叉验证t检验（不同模型-学习器对比）
模型A，B，误差率分别是：$\varepsilon_1^A,\varepsilon_2^A,\varepsilon_3^A,...,\varepsilon_k^A$，$\varepsilon_1^B,\varepsilon_2^B,\varepsilon_3^B,...,\varepsilon_k^B$，其中$\varepsilon_i^A,
\varepsilon_i^B$,是在相同的i折训练/测试集上的结果，则K折可以构建**“成对t检验”**来比较
假设$H_0$：若两个学习器的性能相同，则他们使用相同的训练/测试集得到的错误率应该是相同的，即$\varepsilon_i^A =
\varepsilon_i^B$

构建差值: $\Delta_i = \varepsilon_i^A -
\varepsilon_i^B$，计算他们的$\mu, \sigma^2$,构建t统计量(自由度k-1)：
$$
\tau_t = \left |\frac{\sqrt{k}\mu}{\sigma} \right |
$$
是否拒绝假设，上一节。

**要点**：要做好有效的假设检验，前提是测试误差率均为泛化错误率的独立采样，k折交叉验证，因为数据会有重合，使的测试错误率并不是完全独立的，会导致过高估计假设成立的概率，缓解这一问题，采用5*2折交叉验证，每次2折交叉验证之前，随机打乱数据；
为了缓解非独立性，我们只计算第一次的$\mu = \frac{\Delta_1^1 + \Delta_1^2}{2}$,方差还是正常计算$\sigma_i^2
= (\Delta_i^1 - \frac{\Delta_i^1 + \Delta_i^2}{2})^2 + (\Delta_i^2 -
\frac{\Delta_i^1 + \Delta_i^2}{2})^2$,新的统计量
$$
\tau_t = \frac{\mu}{\sqrt {0.2
\cdot \sum_{i=1}^{5} \sigma_i^2}  }
$$
参考：[T检验](https://wiki.mbalib.com/wiki/T%E6%A3%80%E9%AA%8C)


### McNemar检验
使用留出法，可以得到学习器A，B的测试错误率，还可以获得A，B分类结果的差异，即全部正确，全部错误，1个正确，1个错误的样本数：列联表如下：

| 算法B |算法A    | 算法A    |
| ----- | -------- | -------- |
|       | 正确     | 错误     |
|正确  | $e_{00}$ | $e_{01}$ |
| 错误  | $e_{10}$ | $e_{11}$ |

如果两个学习器性能相同，应该有$e_{10} = e_{01}$,变量$\left| \varepsilon_{01} - \varepsilon_{10}
\right|$应该服从正太分布,所以变量：
$$
\tau_{\chi^2} = \frac{(\left| \varepsilon_{01} -
\varepsilon_{10}\right| - 1)^2}{\varepsilon_{01} + \varepsilon_{10}}
$$
服从自由度1的$\chi^2$的卡方分布。
-1是为了修正，参卡：[为什么McNemar检验的卡方分布要在分子上有一个"-1"?](https://www.zhihu.com/question/47835037)
### Friedman检验与Nemenyi检验

以上检验都是一个数据集比较2个算法，实际中，我们需要比较多个算法。

方法1：同上，算法两两比较
方法2：使用基于排序的算法Friedman

n个数据集，k个算法：

1. 计算算法在每个数据集上的排序
2. 计算算法的平均序值
3.
构造统计量，服从$\chi^2$分布
4. Nemenyi后续检验

# 偏差与方差
对于测试样本，$y_D$：为$x$在数据集中的标记，$y$为$x$的真实标记，通常$E_D[y_D-y]=0$，即噪音期望为0。抽样方法问题，会存在部分的抽样误差，会体现在后续的泛化误差里。

$f(x;D)$为训练集上模型$f$在$x$上的输出。学习算法的期望预测$\bar{f(x)}=E_D[f(x;D)]$

**噪声**使用样本数量相同的不同训练集产生为：$var(x) = E_D[(f(x;D) - \bar{f(x)})^2]$。**噪声**为：$\varepsilon^2=E_D[(y_D - y)^2]$

**偏差（bias）**：期望输出与真实标记的差别，即$bias^2(x) = (\bar{f(x)} - y)^2$



**算法的期望泛化误差**(参考南瓜书)
$$
E(f;D) = E_D[(f(x;D) -
y_D)^2]\\
=bias^2(x) + var(x) + \varepsilon^2
$$
![泛化误差与bias和var](assets/泛化误差与bias和var.png)

> 泛化误差为第一条线
